In [55]:
# -*- coding: UTF-8 -*-
import time
import jieba
from gensim.models import word2vec
from gensim import models
import numpy as np

In [56]:
# 精確模式 ：將句子最精確地切開，叫適合文本分析, cut_all=False
# 全模式：把句子中所有的可以成詞的詞語都掃描出來, 速度快, cut_all=True
# 搜索引擎模式：在精確模式的基礎上對長詞再次切分，提高召回率，適合用於搜尋引擎分詞, jieba.cut_for_search(Content)            
# call jieba api
def jiebaCut(s):
    words = jieba.cut(s, cut_all=False)
    result = removeStopWords(words)
    return result

# remove stopwords
def removeStopWords(words):
    result = []
    for w in words:
        if w not in stopWordsSet:
            result.append(w)
    return result

# define all state 
def state(s,flag):
    nextline = 1
    if s is None or s == "":
        return flag, nextline
    # state: 1, s[0] = C
    if s[0] == 'C':
        flag, nextline = 1, 0
    # state: 2, s[0] = Q
    elif s[0] == 'Q':
        flag, nextline = 2, 0
    # state: 3, s[0] = A
    elif s[0] == 'A':
        flag, nextline = 3, 0
    # state: 4, do jieba cut
    return flag, nextline

In [57]:
def main():
    sTime = time.time()
    print("Start process CQA dataset")
    cNum = 0
    with open('CQA.txt', 'r') as file:
        flag, end = 0, 0
        cList, qList, aList = [],[],[]
        tempC = []
        ans = ""
        for i in file.readlines():
            s = i.strip()
            flag, nextline = state(s,flag)
            # one corpus process done!
            if end == 4:
                cNum +=1
                ans = s
                word2Vec(cList, qList, aList, cNum)
                print("Corpus: %d" % cNum)
                #print("corpus:\n",cList,'\nquestion:\n',qList,'\nanswer:\n',aList,'\ncorrect ans:\n',ans,'\n')
                cList, qList, aList = [],[],[]
                flag, end = 0, 0
                ans = ""
                continue
            # still on state
            if nextline != 1:
                continue
            # on state 1, process Corpus
            elif flag == 1:
                cutRes = jiebaCut(s)
                for c in cutRes:
                    tempC.append(c)
                if nextline == 1:
                    if tempC:
                        cList.append(tempC)
                        tempC = []
            # on state 2, process Question
            elif flag == 2:
                cutRes = jiebaCut(s)
                for c in cutRes:
                    qList.append(c)
            # on state 3, process Answer
            elif flag == 3:
                end += 1
                # example: （B） 吃飯比讀書更為重要 
                tempS = ""
                skip = ['A','B','C','D','（',')']
                check = 0
                for j in s:
                    if check == 3:
                        tempS += j
                    else:
                        check += 1
                tempS = tempS.strip()
                cutRes = jiebaCut(tempS)
                tempL = []
                for c in cutRes:
                    tempL.append(c)
                aList.append(tempL)

        print("\nTotal corpus numbers: %d" % cNum)
        print("Processing all CQA dataset corpus took %.2fs" % (time.time()- sTime))

In [ ]:

def word2Vec(cList, qList, aList, cNum):
    sTime = time.time()
    print("Start process corpus : %d" % (cNum))
    na = np.zeros(250)
    nb = np.zeros(250)
    nc = np.zeros(250)
    count = 0
    for c in cList:
        for w in c:
            try:
                m = model[w]
            except KeyError as e:
                print('this word is not in model.')
                continue
            for n in range(250):
                na[n] += m[n]
            count +=1
                
    for w in qList:
        try:
            m = model[w]
        except KeyError as e:
            print('this word is not in model.')
            continue
        for n in range(250):
            nb[n] += m[n]
        count +=1
        
    for a in aList:
        for w in a:
            try:
                m = model[w]
            except KeyError as e:
                print('this word is not in model.')
                continue
            for n in range(250):
                nc[n] += m[n]
            count +=1
    print("This corpus has %d split words." % (count))
    print("Process all corpus content took %.2fs." % (time.time()- sTime))

In [ ]:

def initial():
    # jieba setting
    relativePath = os.getcwd()
    jieba.set_dictionary(relativePath + '/jieba_setting/dict.txt.big')
    # add user dictionary to improve jieba cut precision
    # jieba.load_userdict(relativePath + '/jieba_setting/yourfile.txt')
    
    # stopwords setting
    stopWordsSet = set()
    with open(relativePath + '/jieba_setting/stopwords.txt', 'r') as stop:
        for i in stop:
            stopWordsSet.add(i.strip('\n'))
    
    # load word2vec model
    sTime = time.time()
    model = models.Word2Vec.load(relativePath + '/wiki/python/word2vec.model')
    print("Load word2vec model success! took %.2fs" % (time.time()-sTime))

if __name__ == "__main__":
    initial()
    main()

Load word2vec model success! took 239.00s
Start process CQA dataset


Building prefix dict from /home/ethan/pythonwork/ipynotebook/MOST/jieba_setting/dict.txt.big ...
Loading model from cache /tmp/jieba.ub845031c377e450e44c5e52006afa1e4.cache
Loading model cost 18.435 seconds.
Prefix dict has been built succesfully.


[['有', '一種', '鳥叫', '蜂鳥', '是', '鳥類', '中', '最小', '的', '鳥', '只有', '拇指', '大小', '這種', '鳥嘴', '型', '細長', '專吃', '花蜜', '和', '花', '上', '的', '小', '昆蟲', '產於', '南美洲', '整日', '盤旋', '於', '仙人掌', '等', '幾種', '花間', '也許', '營養', '太', '單調', '總', '也', '吃', '不大', '飛不高', '而', '有些', '人', '讀書', '做學問', '的', '方式', '和', '蜂鳥', '很', '相似', '這些', '人愛', '讀書', '卻', '不', '大會', '讀書', '往往', '成為', '書', '的', '奴僕', '他們', '埋進', '書堆', '一心', '只讀', '自己', '的', '書', '任', '天塌下來', '也', '不', '管', '這些', '人', '埋頭', '苦讀', '精神可嘉', '本', '無可厚非', '然而', '令人擔心', '的', '是', '鑽', '進', '象牙塔', '出不來', '讀書', '就', '像', '吃飯', '不可', '偏食', '印刷', '的', '書籍', '要讀', '人生', '這本', '大書', '更得讀']]
Start process corpus : 1


/home/ethan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


this word is not in model.
This corpus has 132 split words.
Process all corpus content took 1.36s.
Corpus: 1
[['有', '一種', '鳥叫', '蜂鳥', '是', '鳥類', '中', '最小', '的', '鳥', '只有', '拇指', '大小', '這種', '鳥嘴', '型', '細長', '專吃', '花蜜', '和', '花', '上', '的', '小', '昆蟲', '產於', '南美洲', '整日', '盤旋', '於', '仙人掌', '等', '幾種', '花間', '也許', '營養', '太', '單調', '總', '也', '吃', '不大', '飛不高', '而', '有些', '人', '讀書', '做學問', '的', '方式', '和', '蜂鳥', '很', '相似', '這些', '人愛', '讀書', '卻', '不', '大會', '讀書', '往往', '成為', '書', '的', '奴僕', '他們', '埋進', '書堆', '一心', '只讀', '自己', '的', '書', '任', '天塌下來', '也', '不', '管', '這些', '人', '埋頭', '苦讀', '精神可嘉', '本', '無可厚非', '然而', '令人擔心', '的', '是', '鑽', '進', '象牙塔', '出不來', '讀書', '就', '像', '吃飯', '不可', '偏食', '印刷', '的', '書籍', '要讀', '人生', '這本', '大書', '更得讀']]
Start process corpus : 2
this word is not in model.
This corpus has 125 split words.
Process all corpus content took 0.07s.
Corpus: 2
[['有', '一種', '鳥叫', '蜂鳥', '是', '鳥類', '中', '最小', '的', '鳥', '只有', '拇指', '大小', '這種', '鳥嘴', '型', '細長', '專吃', '花蜜', '和', '花', '上', '的', '小

...

this word is not in model.
this word is not in model.
this word is not in model.
This corpus has 167 split words.
Process all corpus content took 0.41s.
Corpus: 244
[['男', '昨天晚上', '你', '嫂子', '做', '了', '一', '桌子', '的', '菜', '醋溜', '魚片', '醋溜', '白菜', '醋溜', '蝦', '飯後', '還來', '了', '杯', '青梅', '醋', '一', '整晚', '話', '沒', '兩句', '和', '平常', '多話', '的', '她', '判若兩人', '整個', '人', '陰陽怪氣', '的'], ['女', '哥', '我', '昨天', '和', '嫂嫂', '出門', '買', '東西', '在', '捷運', '站', '看見', '你', '和', '一位', '長髮', '女性', '共撐', '一把', '傘', '該', '不會', '是', '…'], ['男', '長髮', '女性', '昨天', '一出', '捷運', '站', '就', '下', '了', '個', '傾盆大雨', '我', '身上', '沒傘', '就', '跟', '附近', '賣傘', '的', '小販', '買', '了', '一把', '那個', '年輕人', '做起', '生意', '來', '可', '積極', '了', '邊說', '他', '還邊', '示範', '沒想到', '那', '時風', '大', '小伙子', '留', '的', '一頭', '長髮', '給傘骨', '纏住', '了', '兩個', '大', '男人', '拉扯', '了', '好', '一會兒', '才', '把', '頭髮', '救', '出來', '不是', '我', '要說', '一個', '大', '男人', '留', '什麼', '長髮', '啊']]
Start process corpus : 245
this word is not in model.
this word is not in model.
this 

This corpus has 190 split words.
Process all corpus content took 0.13s.
Corpus: 259
[['2008', '年', '台灣', '一部', '以', '小鎮風光', '為', '背景', '的', '電影', '讓', '這個', '小鎮', '成', '了', '新', '的', '觀光', '景點', '大量', '遊客', '來', '這裡', '不僅', '創造', '了', '小鎮', '的', '經濟', '價值', '也', '為', '當地', '居民', '增加', '了', '大量', '的', '工作', '機會', '好', '的', '劇本', '好', '的', '演員', '當然', '是', '電影', '成功', '的', '要素', '但', '讓', '人', '好奇', '的', '是', '這個', '小鎮', '怎麼', '透過', '電影', '吸引', '這麼', '多人來', '參觀', '是', '電影', '中', '那', '濃濃的', '人情味', '厚重', '的', '歷史', '氣息', '還是', '讓', '人', '口水直流', '的', '地方', '美食'], ['「', '觀眾', '透過', '電影', '看見', '了', '小鎮', '的', '魅力', '所以', '想', '去', '感受', '感受', '」', '導演', '的話', '點出', '了', '答案', '但是', '電影', '對', '小鎮', '的', '影響', '能', '有', '多久', '是不是', '隨著', '電影', '結束', '這股', '觀', '光熱', '也', '就', '跟', '著', '消失'], ['這部', '電影', '的', '推銷', '手法', '也許', '值得', '飯店', '經營者', '參考', '若能將', '飯店', '與', '當地', '人文', '相互', '結合', '那麼', '被', '電影', '所', '帶動', '的', '觀光', '風氣', '也許', '就', '有', '可能', '持續', '下去']]
Start process corp

This corpus has 171 split words.
Process all corpus content took 0.35s.
Corpus: 269
[['女', '這次', '金融', '海嘯', '使', '一些', '知名', '的', '金融', '集團', '陷入', '空前', '危機', '好幾家', '銀行', '宣布', '倒閉', '這樣', '惡劣', '的', '投資', '環境', '使', '投資者', '失去', '了', '信心', '還有', '人', '醞釀', '集體', '控告', '銀行', '想', '請問', '一下', '林', '經理', '保險業', '會', '受到', '波及', '嗎'], ['男', '應該', '不會', '保險業', '跟', '銀行', '不同', '本來', '就', '不', '可以', '從事', '高', '報酬', '高風險', '的', '投資', '藉', '著', '這次', '的', '金融危機', '剛好', '可以', '讓', '保險市場', '回歸', '基本面'], ['女', '回到', '基本面', '真是', '發人深省', '的話', '近年來', '保單', '的', '價值', '顛倒', '了', '以', '投資', '為', '目的', '的', '商品', '大幅', '成長', '以', '保障', '為', '目的', '的', '商品', '反而', '變少', '了', '好像', '大家', '都', '忘', '了', '保險制度', '最初', '的', '用意', '與', '基本', '的', '價值', '為', '什麼', '會', '這樣', '呢'], ['男', '這', '幾年', '社會', '上', '瀰漫', '著', '一股', '急功近利', '的', '氣息', '拼命', '鼓吹', '投資', '理財', '還有', '奢華', '的', '物質', '生活', '使得', '人人', '都', '想', '一夜', '致富', '所以', '才', '會', '這樣', '吧']]
Start process corpus : 270
This corpus has 180

This corpus has 277 split words.
Process all corpus content took 0.44s.
Corpus: 278
[['以下', '兩人', '正在', '討論', '一部', '電影', '請', '聽', '這段', '談話', '然後', '回答', '四個', '問題'], ['現在', '請', '聽', '對話'], ['男', '我', '昨天', '看', '了', '你', '說', '的', '那部', '電影', '嗯', '…', '我', '覺得', '在', '美術', '攝影', '和', '配樂', '方面', '都', '維持', '著', '李', '導演', '一貫', '的', '高', '水準', '劇情', '的話', '通俗易懂', '如果', '把', '這部', '電影', '跟', '李', '導演', '的', '上', '一部', '作品', '做個', '對照', '不難', '發現', '情節', '安排', '上', '有', '許多', '相似之處'], ['女', '哇', '很', '專業', '嘛', '但', '大部分', '人', '在', '欣賞', '電影', '時', '常常', '是', '先', '注意', '到', '演員', '的', '演技', '你', '覺得', '幾位', '主角', '…'], ['男', '哈', '我', '正想', '接著', '說', '演員', '們', '精湛', '的', '演技', '可以', '說', '是', '這部', '片', '的', '賣點', '之一', '像', '飾演', '男主角', '的', '李文', '在', '面對', '女主角', '來訪', '時', '那種', '強裝', '鎮定', '但', '其實', '手足無措', '的', '神態', '演活', '了', '這個', '角色', '如', '孩子', '般', '純真', '的', '一面', '如果', '比對', '一下', '他', '這', '幾年', '參與', '演出', '的', '電影', '其實', '就', '能', '看出', '他', '的', '演技', '已經', 

this word is not in model.
This corpus has 295 split words.
Process all corpus content took 0.04s.
Corpus: 285
[['「', '基因治療', '」', '是', '指將', '正常', '健康', '的', '基因', '移植', '到', '病患', '有', '缺失', '或是', '功能', '不全', '的', '基因', '中', '藉此', '治癒', '癌症', '或', '遺傳疾病', '等', '重大', '疾病', '以下', '是', '一段', '關於', '基因治療', '的', '討論', '請', '聽', '完', '後', '回答', '四個', '問題'], ['現在', '請', '聽', '對話'], ['男', '跟', '妳', '說', '個', '振奮人心', '的', '消息', '新', '的', '基因', '療法', '正', '準備', '進入', '臨床', '試驗', '哎', '等', '了', '這麼', '多年', '歷經', '了', '多少', '的', '研究', '基因治療', '總算', '要成', '為', '革命性', '的', '醫學', '療法', '了'], ['女', '嗯', '…', '這個', '消息', '對', '人類', '而言', '算不算', '是則', '福音', '還有', '待', '商榷', '吧', '畢竟', '治療', '後', '的', '基因', '在', '演化', '上會', '起', '什麼', '作用', '無人', '可知', '貝格', '博士', '就', '說', '了', '基因治療', '的', '同時', '也', '可能', '剔除', '掉', '一些', '對', '人類', '而言', '重要', '的', '基因', '像是', '鐮刀', '型', '貧血', '的', '基因', '缺陷', '其實', '對', '瘧疾', '就', '具有', '免疫力', '所以', '執行', '基因', '療法', '前', '實在', '得', '謹慎', '思考', '一旦', '改變', '了', '人